In [ ]:
import numpy as np
from scipy.signal import welch
import matplotlib.pyplot as plt
import seaborn as sns
import sys
if not '..' in sys.path:
    sys.path.append('..')
from pfcommon import OU_2

In [ ]:
tend = 100000
dt = 2e-3
mean,stddev,tau = 0,0.01*322,20e-3
μ,c,α = mean,stddev*np.sqrt(2/tau),1/tau
# μ,c,α = 0,1,10*2*np.pi
# mean,stddev,tau = μ,np.sqrt(c**2/(2*α)),1/α
time = np.r_[0 : tend : dt]
N = time.size
cutoff = α/(2*np.pi)
x = OU_2(dt, α, μ, c, N)

In [ ]:
window = 100 / dt
onesided = True
freq,P = welch(x, 1/dt, window='boxcar', nperseg=window, noverlap=window/4,
               nfft=window, return_onesided=onesided, scaling='density')
if onesided:
    P /= 2
else:
    Nf = freq.size
    freq = freq[:Nf//2]
    P = P[:Nf//2]
P_theor = (c/α)**2 / (1 + (2*np.pi*freq/α)**2)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(5,3))
ax.plot(freq, P, 'k', lw=0.75)
ax.plot(freq, P_theor, 'r', lw=2)
ax.plot(cutoff+np.zeros(2), [0, (c/α)**2], 'g--', lw=1)
ax.set_xscale('log')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('PSD')
sns.despine()
fig.tight_layout()

The variance of a process is the integral of its PSD over all frequencies:

In [ ]:
from scipy.integrate import simpson
var = simpson(P_theor, freq)
if onesided:
    var *= 2
print(f'Theoretical variance: {var:.4f}.')
print(f'Numerically computed variance: {x.var():.4f}.')